### Change-log

- No need to pass 'soup' as arguments to function
- Added source, modified_date

### Import Libraries

In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from Levenshtein import distance
from datetime import datetime 

### Loading Dataset and Data Preprocessing

In [2]:
def load_df(filename):
    return pd.read_csv(f'./datasets/{filename}.csv')

layout1 = load_df('ABC_layout_1')
layout2 = load_df('PQR_layout_2')
layout3 = load_df('layout_3_voters')
layout4 = load_df('KLM_layout_4')
layout5 = load_df('layout_5_license')

layout1 = layout1.rename(columns={"First Name": "Name", "Father Name": "Father_Name", "Permanent_Adress":"Permanent_Address"})
layout2 = layout2.rename(columns = {"Customer_ID": "Mobile Number"})
layout3 = layout3.rename(columns={"votersName": "Name", "votersFatherName": "Father_Name", "votersMotherName": "Mother Name", " Gender": "Gender", "Permanent_Adress":"Permanent_Address"})
layout4 = layout4.rename(columns={"Father Name": "Father_Name"})

layouts = [layout1, layout2, layout3, layout4, layout5]
layout_sources = ['bank', 'esewa', 'voter', 'electricity', 'license']

for layout, source in zip(layouts, layout_sources):
    layout['source'] = source
    layout['modified_date'] = datetime.now()

def sanitize(df):
    return df.map(lambda x: x.replace(',', '').replace(' ', '').strip() if isinstance(x, str) else '' if pd.isna(x) else x)
    
def create_soup(df, df_, soup, soup_name):
    df[soup_name] = df_[soup].apply(lambda x: ' '.join(x.values.astype(str)).lower(), axis=1)

layout_copies = [layout.copy() for layout in layouts]
soup = ['Name', 'Date of Birth', 'Father_Name']

for layout, layout_copy, in zip(layouts, layout_copies):
    layout_copy = sanitize(layout_copy)
    create_soup(layout, layout_copy, soup, "soup")

### Entity Matching

In [3]:
def combine_layouts(A, B, metric = 'cosine', threshold=0.8):
    if metric == 'cosine':
        tfidf = TfidfVectorizer(stop_words='english')
        
        combined_soup = pd.concat([A['soup'], B['soup']], ignore_index=True)
        tfidf.fit(combined_soup)
        
        tfidf_matrix_A = tfidf.transform(A['soup'])
        tfidf_matrix_B = tfidf.transform(B['soup'])
        
        similarity = cosine_similarity(tfidf_matrix_A, tfidf_matrix_B)
        similarity_df = pd.DataFrame(similarity, index=A.index, columns=B.index)

        max_idx_row = similarity_df.idxmax(axis=1)
        similarity_mask = similarity_df.max(axis=1) > threshold
        
    elif metric == 'levenshtein':
        distance_matrix = pd.DataFrame(np.zeros((len(A), len(B))), index=A.index, columns=B.index)

        for i in A.index:
            for j in B.index:
                distance_matrix.loc[i, j] = distance(A.loc[i, soup_A], B.loc[j, soup_B])

        min_idx_row = distance_matrix.idxmin(axis=1)
        min_distance = distance_matrix.min(axis=1)

    # Initialize the combined DataFrame with columns from both DataFrames
    combined_columns = list(set(A.columns) | set(B.columns))
    combined_data = pd.DataFrame(columns=combined_columns)
    
    # Merge the similar rows
    for idx_A in A.index:
        if similarity_mask[idx_A]:
            idx_B = max_idx_row[idx_A]
            combined_row = A.loc[idx_A].combine_first(B.loc[idx_B])
            combined_row['source'] = A.loc[idx_A]['source'] + ', ' + B.loc[idx_B]['source'] 
            combined_row['modified_date'] = datetime.now()
        else:
            combined_row = A.loc[idx_A]
        combined_data = pd.concat([combined_data, combined_row.to_frame().T], ignore_index=True)
    
    new_records = B.loc[~B.index.isin(max_idx_row[similarity_mask].values)]
    combined_data = pd.concat([combined_data, new_records], ignore_index=True)
    return combined_data

result_12 = combine_layouts(layout1, layout2)
result_123 = combine_layouts(result_12, layout3)
result_1234 = combine_layouts(result_123, layout4)
final_result = combine_layouts(result_1234, layout5)
final_result

,votersAge,Name,soup,PAN_Number,Gender,Customer Code,modified_date,SC Number,License Number,votersID,...,National Id,Father_Name,Permanent_Address,SpouseName,Mobile Number,Citizenship Number,Blood Group,Mother Name,Temporary_Address,Customer ID
0,45,Ram Thapa,ramthapa 1990-01-01 rambahadurthapa,ABCDE1234F,Male,21216874,2024-05-24 00:35:00.378111,001.01.01,15-05-58353205,11116874,...,AB123C,Ram Bahadur Thapa,"Baluwatar, Kathmandu, Nepal",Sita Thapa,1234567890,624-93227-32431/660086,AB+,Laxmi Thapa,"Gongabu, Kathmandu, Nepal",3245
1,38,Sita Shrestha,sitashrestha 1991-02-02 hariprasadshrestha,FGHIJ5678K,Female,22359363,2024-05-24 00:35:00.378111,001.01.02,21-08-00435579,22259363,...,DE456F,Hari Prasad Shrestha,"Lakeside, Pokhara, Nepal",Ravi Sharma,2345678901,747-42087-31417/584714,AB-,Radha Sharma,"New Road, Pokhara, Nepal",3246
2,52,Hari Gurung,harigurung 1992-03-03 gopalkrishnagurung,LMNOP9012L,Male,33485241,2024-05-24 00:35:00.378111,001.01.03,93-12-35351480,33385241,...,GH789I,Gopal Krishna Gurung,"Chitwan National Park, Chitwan, Nepal",Maya Adhikari,3456789012,389-45382-93886/821590,B-,Gita Adhikari,"Pulchowk, Lalitpur, Nepal",3247
3,30,Gita Tamang,gitatamang 1993-04-04 shyamlaltamang,QRSTU3456M,Female,45475489,2024-05-24 00:35:00.378111,001.01.04,65-03-68139881,44475489,...,JK012L,Shyam Lal Tamang,"Biratnagar, Morang, Nepal",Surya Rai,4567890123,571-38785-99733/440035,A-,Mina Rai,"Bagbazar, Kathmandu, Nepal",3248
4,27,Mohan Lama,mohanlama 1994-05-05 krishnarajlama,VWXYZ7890N,Male,56562139,2024-05-24 00:35:00.345560,001.01.05,NaN,55562139,...,MN345O,Krishna Raj Lama,"Bharatpur, Chitwan, Nepal",Sarita Karki,5678901234,NaN,NaN,Kalpana Karki,NaN,3249
5,41,Radha Magar,radhamagar 1995-06-06 narayankumarmagar,ABCD1234PQ,Female,67698214,2024-05-24 00:35:00.378111,001.01.06,82-09-81734599,66698214,...,PQ678R,Narayan Kumar Magar,"Butwal, Rupandehi, Nepal",Pemba Gurung,6789012345,033-51347-62581/380746,B-,Nima Gurung,"Chabahil, Kathmandu, Nepal",3250
6,36,Krishna Rai,krishnarai 1996-07-07 govindabahadurrai,EFGH5678RS,Male,78714635,2024-05-24 00:35:00.353177,001.01.07,NaN,77714635,...,ST901U,Govinda Bahadur Rai,"Hetauda, Makwanpur, Nepal",Laxmi Shrestha,7890123456,NaN,NaN,Saru Shrestha,"Kumaripati, Lalitpur, Nepal",3251
7,50,Sarita Sherpa,saritasherpa 1997-08-08 shivanarayansherpa,TUVW9012XY,Female,89847326,2024-05-24 00:35:00.393126,001.01.08,98-03-72394228,88847326,...,VW234X,Shiva Narayan Sherpa,"Janakpur, Dhanusa, Nepal",Raj Maharjan,8901234567,323-37869-95909/623481,A-,Nanu Maharjan,"New Baneshwor, Kathmandu, Nepal",3252
8,43,Bikash Karki,bikashkarki 1998-09-09 bhagirathbahadurkarki,ZABC3456DE,Male,91953421,2024-05-24 00:35:00.353177,001.01.09,NaN,99953421,...,YZ567A,Bhagirath Bahadur Karki,"Nepalgunj, Banke, Nepal",Pema Tamang,9012345678,NaN,NaN,Lhamu Tamang,"Boudha, Kathmandu, Nepal",3253
9,34,Nisha Bhandari,nishabhandari 1999-10-10 suryabahadurbhandari,FGHI7890JK,Female,10203847,2024-05-24 00:35:00.395294,001.01.10,42-08-02614125,10103847,...,BC890D,Surya Bahadur Bhandari,"Dharan, Sunsari, Nepal",Manish KC,123456789,272-28301-42325/881177,A-,Lila KC,"Kalanki, Kathmandu, Nepal",3254
